<a href="https://colab.research.google.com/github/olympus999/Carvana-Image-Masking-Challenge/blob/master/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install ijson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
import urllib.request
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.types import *
from os.path import isfile
import zipfile
import json
import ast
import time
import ijson

In [3]:
conf = SparkConf().set("spark.driver.memory", "10g")\
                  .set("spark.executor.memory", "10g")
spark = SparkSession.builder.config(conf=conf).master("local[*]").appName('avaandmed_ariregister').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [4]:
# spark.sparkContext.getConf().getAll()

In [5]:
folder_data = 'data'
folder_data_unpacked = os.path.join(folder_data, 'unpacked')
folder_data_spark_json = os.path.join(folder_data_unpacked, 'spark_json')

# Data is downloaded from here: https://avaandmed.ariregister.rik.ee/et/avaandmete-allalaadimine#ettepanekud-ja-tagasiside
generic_data = {
    'lihtandmed': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__lihtandmed.csv.zip',
    # 'yldandmed': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__yldandmed.json.zip',
    # 'registrikaardid': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__registrikaardid.json.zip',
    # 'kaardile_kantud_isikud': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__kaardile_kantud_isikud.json.zip',
    # 'kandevalised_isikud': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__kandevalised_isikud.json.zip',
    # 'osanikud': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__osanikud.json.zip',
    # 'kasusaajad': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__kasusaajad.json.zip',
    'kommertspandiandmed': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__kommertspandid.json.zip',
    # 'maarused': 'https://avaandmed.ariregister.rik.ee/sites/default/files/avaandmed/ettevotja_rekvisiidid__maarused.json.zip'
}

yearly_data = {
    # 'aruannete_yldandmed': 'https://avaandmed.ariregister.rik.ee/sites/default/files/1.aruannete_yldandmed_kuni_31102023.zip',
    'emtak_myygitulu_jaotus': 'https://avaandmed.ariregister.rik.ee/sites/default/files/2.EMTAK_myygitulu_kuni_31102023.zip',
    # 'myygitulu_geograafiline_jaotus': 'https://avaandmed.ariregister.rik.ee/sites/default/files/3.myygitulu_geograafiline_kuni_31102023.zip',
    # 'pohinaitajad_2020_aasta_aruannetest': 'https://avaandmed.ariregister.rik.ee/sites/default/files/4.2020_aruannete_elemendid_kuni_31102023.zip',
    # 'pohinaitajad_2021_aasta_aruannetest': 'https://avaandmed.ariregister.rik.ee/sites/default/files/4.2021_aruannete_elemendid_kuni_31102023.zip',
    # 'pohinaitajad_2022_aasta_aruannetest': 'https://avaandmed.ariregister.rik.ee/sites/default/files/4.2022_aruannete_elemendid_kuni_31102023.zip',
}

# Data is downloaded to Google Drive as they only keep the last 3 years of data available on their site.
# Data is downloaded from here: https://www.emta.ee/ariklient/amet-uudised-ja-kontakt/uudised-pressiinfo-statistika/statistika-ja-avaandmed#tasutud-maksud-failid

paid_taxes = {
    'tasutud_maksud_2020_i_kvartal': 'https://drive.google.com/uc?id=1h1-hW7sTCMlOxxvz4258W403_HC5QTJV',
    'tasutud_maksud_2020_ii_kvartal': 'https://drive.google.com/uc?id=1qDBr9U8WtSsFucuyEkJy-Hmhx2222zpY',
    'tasutud_maksud_2020_iii_kvartal': 'https://drive.google.com/uc?id=1LFo2bueoIcedmDUdbN6YLZN0_53HWxFY',
    'tasutud_maksud_2020_iv_kvartal': 'https://drive.google.com/uc?id=1OIU9IlYQlRtzsccOSqB6uEHLBnq4Axst',
    'tasutud_maksud_2021_i_kvartal': 'https://drive.google.com/uc?id=15t26vju1g3wR1Bqw7QCe7VFSIUXpgAWu',
    'tasutud_maksud_2021_ii_kvartal': 'https://drive.google.com/uc?id=1Rz6KmBzw61T98noX4_aRWiZEiAjGUSa3',
    'tasutud_maksud_2021_iii_kvartal': 'https://drive.google.com/uc?id=1aOJ_-KQDCs-oIFZMLqJql4IkICo5P-lk',
    'tasutud_maksud_2021_iv_kvartal': 'https://drive.google.com/uc?id=10XyphD7LCub2B9EySdLG47KXhrJr9ipD',
    'tasutud_maksud_2022_i_kvartal': 'https://drive.google.com/uc?id=1sU5wguPYaY2odNMM8aQRwopwm-sjL1oa',
    'tasutud_maksud_2022_ii_kvartal': 'https://drive.google.com/uc?id=1diLKS7NhlLZg2e-c_wRYPLTTUAeTdJxP',
    'tasutud_maksud_2022_iii_kvartal': 'https://drive.google.com/uc?id=1j0aeqXmzG3gdmcJkAZOvC_T_TfXnjNhA',
    'tasutud_maksud_2022_iv_kvartal': 'https://drive.google.com/uc?id=1TGiL1zc_WPSTai8DrbxXSLaoVWXg11Qm',
    'tasutud_maksud_2023_i_kvartal': 'https://drive.google.com/uc?id=1gsCyYuKYjN-nP-NdYWCYuMQZXRU8hpV6',
    'tasutud_maksud_2023_ii_kvartal': 'https://drive.google.com/uc?id=1sskcbN0cAIaD8H4_34AyRxv2AwJliF1Q',
    'tasutud_maksud_2023_iii_kvartal': 'https://drive.google.com/uc?id=1DPVmFwe6BkjD_j_VegXMJaLoyR7z9pcu',
}

all_data = {**generic_data, **yearly_data, **paid_taxes}

In [6]:
# customSchema = StructType([
#     StructField("IDGC", StringType(), True),
#     StructField("SEARCHNAME", StringType(), True),
#     StructField("PRICE", DoubleType(), True)
# ])

key_taxes = 'tasutud_maksud'

schemas = {
  key_taxes :
    StructType([
        StructField("Riiklikud Maksud", DoubleType(), True)
    ])
}

In [17]:
def mkdir(p):
  if not os.path.isdir(p):
    os.makedirs(p)

def is_key_taxes(key: str):
  return key.startswith(key_taxes)

def get_filename(url: str, key: str = None) -> str:
  if key is not None and is_key_taxes(key):
    return key + '.csv'
  return url.split('/')[-1]

def save_avaandmed(key: str, url: str, overwrite=False) -> str:
  "CSV files can be saved to folder ´unpacked´"
  file_name = get_filename(url, key)
  if file_name.endswith('.csv'):
    file_path_name = f"{folder_data_unpacked}/{file_name}"
  else:
    file_path_name = f"{folder_data}/{file_name}"
  if overwrite or not os.path.isfile(file_path_name):
    urllib.request.urlretrieve(url, file_path_name)

  return file_path_name

def file_path_unpacked(key: str) -> str:
  url = all_data[key]
  file_name = get_filename(url, key)
  head, _sep, tail = file_name.rpartition('.zip')
  path = os.path.join(folder_data_unpacked, (head + tail))

  if not is_json_or_csv(path):
    path = path + '.csv' # ´.csv´ is missing in some cases

  return path

def file_path_spark_json(key: str) -> str:
  url = all_data[key]
  file_name = get_filename(url)
  head, _sep, tail = file_name.rpartition('.zip')

  return os.path.join(folder_data_spark_json, (head + tail))

def create_dirs():
  mkdir(folder_data)
  mkdir(folder_data_unpacked)
  mkdir(folder_data_spark_json)

def download_files():
  for key, value in all_data.items():
    print('downloaded', key, '-', save_avaandmed(key, value))

def unpack_files():
  for f in os.listdir(folder_data):
    p = os.path.join(folder_data, f)
    if not os.path.isdir(p):
      with zipfile.ZipFile(p, "r") as zip_ref:
        zip_ref.extractall(folder_data_unpacked)
        print('unpacked', '-', p, 'to', folder_data_unpacked)

def json_to_table(path: str, key: str):
  df = spark.read.json(path)
  create_table(df, key)

def csv_to_table(path: str, key: str):
  kwargs = {
      'path': path,
      'encoding':'iso8859-1',
      'header':True,
      'inferSchema':True,
      'sep':';'
  }

  if is_key_taxes(key):
    pass
    # kwargs['schema'] = schemas[key_taxes]
  df = spark.read.csv(**kwargs)
  create_table(df, key)

def create_table(df, key):
  print(f'Trying to create table ´{key}´ - ', end="")
  df.createOrReplaceTempView(key)
  print(f'Done!')

def is_json(path: str) -> bool:
  return path.endswith('.json')

def is_csv(path: str) -> bool:
  return path.endswith('.csv')

def is_json_or_csv(path: str) -> bool:
  return is_json(path) or is_csv(path)

def modify_json_for_spark(path: str, path_spark: str):
  # Spark expectes certain JSON format.
  # The function changes the following:
  ## [
  ##   { [record] },
  ##   { [record] }
  ## ]
  # to
  ## { [record] }
  ## { [record] }

  if is_json(path):
    print('Starting to modify JSON ', path, end="")
    start_time = time.time()
    temp_path = 'temp.json'

    # Parsing with ijson because some files are quite big in size.
    with open(path, 'r') as f:
      with open(path_spark, 'w') as json_objects:
        for n in ijson.items(f, 'item', use_float=True):
          json_objects.write(json.dumps(n) + '\n')

    time_it_took = time.time() - start_time
    print(' - done! - Time:', round(time_it_took, 1), 'seconds')
  else:
    print('NOT JSON path:', path)

def modify_downloaded_jsons():
  for key in all_data.keys():
    path = file_path_unpacked(key)
    path_spark = file_path_spark_json(key)
    modify_json_for_spark(path, path_spark)

def create_temp_tables():
  for key in all_data.keys():
    path = file_path_unpacked(key)
    if path.endswith('.json'):
      path = file_path_spark_json(key)
      json_to_table(path, key)
    elif path.endswith('.csv'):
      csv_to_table(path, key)

In [18]:
key = 'tasutud_maksud_2020_i_kvartal'
path = file_path_unpacked(key)
csv_to_table(path, key)

Trying to create table ´tasutud_maksud_2020_i_kvartal´ - Done!


In [10]:
# create_dirs()

# download_files()

# unpack_files()

In [11]:
create_dirs()

download_files()

unpack_files()

modify_downloaded_jsons()

create_temp_tables()

downloaded lihtandmed - data/ettevotja_rekvisiidid__lihtandmed.csv.zip
downloaded kommertspandiandmed - data/ettevotja_rekvisiidid__kommertspandid.json.zip
downloaded emtak_myygitulu_jaotus - data/2.EMTAK_myygitulu_kuni_31102023.zip
downloaded tasutud_maksud_2020_i_kvartal - data/unpacked/tasutud_maksud_2020_i_kvartal.csv
downloaded tasutud_maksud_2020_ii_kvartal - data/unpacked/tasutud_maksud_2020_ii_kvartal.csv
downloaded tasutud_maksud_2020_iii_kvartal - data/unpacked/tasutud_maksud_2020_iii_kvartal.csv
downloaded tasutud_maksud_2020_iv_kvartal - data/unpacked/tasutud_maksud_2020_iv_kvartal.csv
downloaded tasutud_maksud_2021_i_kvartal - data/unpacked/tasutud_maksud_2021_i_kvartal.csv
downloaded tasutud_maksud_2021_ii_kvartal - data/unpacked/tasutud_maksud_2021_ii_kvartal.csv
downloaded tasutud_maksud_2021_iii_kvartal - data/unpacked/tasutud_maksud_2021_iii_kvartal.csv
downloaded tasutud_maksud_2021_iv_kvartal - data/unpacked/tasutud_maksud_2021_iv_kvartal.csv
downloaded tasutud_maks

In [ ]:
# def fix_file_endings():
#   "Some files are missing endings. All of them are .csv so we need to fix it"

#   for key in all_data.keys():
#     path = file_path_unpacked(key)
#     if not is_json_or_csv(path):
#       new_path = path + '.csv'
#       print('Fixing file ending for', path)
#       !mv "$path" "$new_path"
#       # os.rename(path, new_path)
#       print('New file', new_path)

In [ ]:
for key in all_data.keys():
  path = file_path_unpacked(key)
  print(path, os.path.isfile(path))

data/unpacked/ettevotja_rekvisiidid__lihtandmed.csv True
data/unpacked/ettevotja_rekvisiidid__yldandmed.json True
data/unpacked/ettevotja_rekvisiidid__registrikaardid.json True
data/unpacked/ettevotja_rekvisiidid__kaardile_kantud_isikud.json True
data/unpacked/ettevotja_rekvisiidid__kandevalised_isikud.json True
data/unpacked/ettevotja_rekvisiidid__osanikud.json True
data/unpacked/ettevotja_rekvisiidid__kasusaajad.json True
data/unpacked/ettevotja_rekvisiidid__kommertspandid.json True
data/unpacked/ettevotja_rekvisiidid__maarused.json True
data/unpacked/1.aruannete_yldandmed_kuni_31102023.csv True
data/unpacked/2.EMTAK_myygitulu_kuni_31102023.csv True
data/unpacked/3.myygitulu_geograafiline_kuni_31102023.csv True
data/unpacked/4.2020_aruannete_elemendid_kuni_31102023.csv True
data/unpacked/4.2021_aruannete_elemendid_kuni_31102023.csv True
data/unpacked/4.2022_aruannete_elemendid_kuni_31102023.csv True


In [ ]:
fix_file_endings()

Fixing file ending for data/unpacked/1.aruannete_yldandmed_kuni_31102023
mv: cannot stat 'data/unpacked/1.aruannete_yldandmed_kuni_31102023': No such file or directory
New file data/unpacked/1.aruannete_yldandmed_kuni_31102023.csv
Fixing file ending for data/unpacked/2.EMTAK_myygitulu_kuni_31102023
mv: cannot stat 'data/unpacked/2.EMTAK_myygitulu_kuni_31102023': No such file or directory
New file data/unpacked/2.EMTAK_myygitulu_kuni_31102023.csv
Fixing file ending for data/unpacked/3.myygitulu_geograafiline_kuni_31102023
mv: cannot stat 'data/unpacked/3.myygitulu_geograafiline_kuni_31102023': No such file or directory
New file data/unpacked/3.myygitulu_geograafiline_kuni_31102023.csv
Fixing file ending for data/unpacked/4.2020_aruannete_elemendid_kuni_31102023
mv: cannot stat 'data/unpacked/4.2020_aruannete_elemendid_kuni_31102023': No such file or directory
New file data/unpacked/4.2020_aruannete_elemendid_kuni_31102023.csv
Fixing file ending for data/unpacked/4.2021_aruannete_elemend

In [ ]:
for key in all_data.keys():
  path = file_path_unpacked(key)
  if not is_json_or_csv(path):
    print(path)

data/unpacked/1.aruannete_yldandmed_kuni_31102023
data/unpacked/2.EMTAK_myygitulu_kuni_31102023
data/unpacked/3.myygitulu_geograafiline_kuni_31102023
data/unpacked/4.2020_aruannete_elemendid_kuni_31102023
data/unpacked/4.2021_aruannete_elemendid_kuni_31102023
data/unpacked/4.2022_aruannete_elemendid_kuni_31102023


### EMTAK codes

In [ ]:
q = """
  WITH extracted_emtak AS (
    SELECT
      element_at(yldandmed.teatatud_tegevusalad.emtak_kood, 1) AS emtak_kood,
      element_at(yldandmed.teatatud_tegevusalad.emtak_tekstina, 1) AS emtak_tekstina
    FROM yldandmed
    WHERE array_size(yldandmed.teatatud_tegevusalad) == 1),
  unique_emtak AS (
    SELECT DISTINCT emtak_kood, emtak_tekstina, count(*) as count
    FROM extracted_emtak
    GROUP BY emtak_kood, emtak_tekstina)
  SELECT * FROM unique_emtak ORDER BY count DESC
      """
emtak_codes = spark.sql(q).cache()

In [ ]:
emtak_codes.show(20, truncate=False)

+----------+-----------------------------------------------------------------------------+-----+
|emtak_kood|emtak_tekstina                                                               |count|
+----------+-----------------------------------------------------------------------------+-----+
|70221     |Ärinõustamine jm juhtimisalane nõustamine                                    |12671|
|68321     |Hoonete ja üürimajade haldus (korteriühistud, elamuühistud, hooneühistud jms)|12289|
|68201     |Enda või renditud kinnisvara üürileandmine ja käitus                         |11986|
|41201     |Elamute ja mitteeluhoonete ehitus                                            |9989 |
|62011     |Programmeerimine                                                             |9715 |
|96021     |Juuksuri- ja muu iluteenindus                                                |8178 |
|47911     |Jaemüük posti või Interneti teel                                             |6743 |
|69202     |Raamatupidamine, m

### Filter companies and get details

In [ ]:
q = """
  WITH selected_companies AS (
    SELECT
      yldandmed.ariregistri_kood as ariregistri_kood,
      yldandmed.teatatud_tegevusalad.emtak_kood as emtak_kood,
      yldandmed.teatatud_tegevusalad.emtak_tekstina as emtak_tekstina,
      yldandmed.aadressid[0].ehak_nimetus as ehak_nimetus, -- As of currently ´yldandmed.aadressid´ always contains 1 address
      yldandmed.aadressid[0].riik as riik,
      yldandmed.aadressid[0].riik_tekstina as riik_tekstina,
      array_compact(
        TRANSFORM(yldandmed.sidevahendid, elem ->
          CASE
            WHEN elem.liik_tekstina = 'Mobiiltelefon' THEN elem.sisu
            ELSE NULL
          END
      )) AS mobiiltelefon, -- In some cases there are multiple contact numbers for example.
      array_compact(
        TRANSFORM(yldandmed.sidevahendid, elem ->
          CASE
            WHEN elem.liik_tekstina = 'Elektronposti aadress' THEN elem.sisu
            ELSE NULL
          END
      )) AS email
    FROM yldandmed
    WHERE array_contains(yldandmed.teatatud_tegevusalad.emtak_kood, '62011')
    and array_size(yldandmed.teatatud_tegevusalad.emtak_kood) = 1
    limit 10
  )
  SELECT * FROM selected_companies
"""

filtered_companies = spark.sql(q).cache()
filtered_companies.show(20, truncate=False)

+----------------+----------+------------------+--------------------------------------------+----+-------------+---------------+--------------------------+
|ariregistri_kood|emtak_kood|emtak_tekstina    |ehak_nimetus                                |riik|riik_tekstina|mobiiltelefon  |email                     |
+----------------+----------+------------------+--------------------------------------------+----+-------------+---------------+--------------------------+
|11728851        |[62011]   |[Programmeerimine]|Kesklinna linnaosa, Tallinn, Harju maakond  |EST |Eesti        |[]             |[jeepld@gmail.com]        |
|16739552        |[62011]   |[Programmeerimine]|Kesklinna linnaosa, Tallinn, Harju maakond  |EST |Eesti        |[]             |[info@044.ai]             |
|14809610        |[62011]   |[Programmeerimine]|Haabneeme alevik, Viimsi vald, Harju maakond|NULL|NULL         |[+372 5541173] |[0x00@bolshakov.ee]       |
|16508930        |[62011]   |[Programmeerimine]|Lasnamäe linnaos

### paid taxes

In [19]:
q = """
  SELECT * FROM tasutud_maksud_2020_i_kvartal limit 10
"""
res = spark.sql(q).cache()
res.show(10, truncate=False)

+------------+----------------------------+--------+--------------------------------------------------+-----------------------------------------------------------------------+--------------------------+----------------+----------------------+------------+---------+
|Registrikood|Nimi                        |Liik    |Registreeritud käibemaksukohustuslaste registrisse|EMTAK tegevusvaldkond, mis on EMTAKi struktuuris tähistatud tähtkoodiga|Maakond                   |Riiklikud Maksud|Tööjõumaksud Ja Maksed|Kaive       |Tootajaid|
+------------+----------------------------+--------+--------------------------------------------------+-----------------------------------------------------------------------+--------------------------+----------------+----------------------+------------+---------+
|10000018    |AMSERV AUTO AS              |Äriühing|jah                                               |HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT        |Harju ( Tallinn )         |

In [21]:
res.printSchema()

root
 |-- Registrikood: string (nullable = true)
 |-- Nimi: string (nullable = true)
 |-- Liik: string (nullable = true)
 |-- Registreeritud käibemaksukohustuslaste registrisse: string (nullable = true)
 |-- EMTAK tegevusvaldkond, mis on EMTAKi struktuuris tähistatud tähtkoodiga: string (nullable = true)
 |-- Maakond: string (nullable = true)
 |-- Riiklikud Maksud: string (nullable = true)
 |-- Tööjõumaksud Ja Maksed: string (nullable = true)
 |-- Kaive: string (nullable = true)
 |-- Tootajaid: integer (nullable = true)



In [ ]:
schemas = {
  key_taxes :
    StructType([
        StructField("Riiklikud Maksud", DoubleType(), True)
    ])
}

In [ ]:
customSchema = StructType([
    StructField("Riiklikud Maksud", DoubleType(), True)
])

In [12]:
spark.sql("SHOW TABLES").show(truncate=False)

+---------+-----------------------------+-----------+
|namespace|tableName                    |isTemporary|
+---------+-----------------------------+-----------+
|         |emtak_myygitulu_jaotus       |true       |
|         |kommertspandiandmed          |true       |
|         |lihtandmed                   |true       |
|         |tasutud_maksud_2020_i_kvartal|true       |
+---------+-----------------------------+-----------+



In [ ]:
spark.sql("DESCRIBE kaardile_kantud_isikud").show(truncate=False)

+-----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                           |data_type                                                                                                                                                                                                                                                         

In [ ]:
q = """
  SELECT * FROM registrikaardid limit 1
"""
schema = spark.sql(q).cache()
schema.printSchema()

root
 |-- ariregistri_kood: long (nullable = true)
 |-- nimi: string (nullable = true)
 |-- registrikaardid: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- kaardi_nr: long (nullable = true)
 |    |    |-- kaardi_piirkond: long (nullable = true)
 |    |    |-- kaardi_tyyp: string (nullable = true)
 |    |    |-- kanded: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- kande_nr: long (nullable = true)
 |    |    |    |    |-- kandeliik: string (nullable = true)
 |    |    |    |    |-- kandeliik_tekstina: string (nullable = true)
 |    |    |    |    |-- kpv: string (nullable = true)



In [ ]:
schema.show(truncate=False)

+----------------+---------------+------------------------------------------------------------------------------------------------------------------+
|ariregistri_kood|nimi           |registrikaardid                                                                                                   |
+----------------+---------------+------------------------------------------------------------------------------------------------------------------+
|16372442        |000 Holdings OÜ|[{1, 5, R, [{1, 2, Esmakanne, 23.11.2021}, {2, 12, Muutmiskanne, 01.02.2023}, {3, 12, Muutmiskanne, 01.09.2023}]}]|
+----------------+---------------+------------------------------------------------------------------------------------------------------------------+



In [ ]:
filtered_companies.count()

10562

In [ ]:
filtered_companies.printSchema()

root
 |-- ariregistri_kood: long (nullable = true)
 |-- emtak_kood: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- emtak_tekstina: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ehak_nimetus: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- riik: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- mobiiltelefon: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- email: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
q = """
  SELECT * FROM yldandmed limit 1
"""
yldandmed_schema = spark.sql(q).cache()
yldandmed_schema.printSchema()

root
 |-- ariregistri_kood: long (nullable = true)
 |-- nimi: string (nullable = true)
 |-- yldandmed: struct (nullable = true)
 |    |-- aadressid: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- aadress_ads__adob_id: string (nullable = true)
 |    |    |    |-- aadress_ads__adr_id: long (nullable = true)
 |    |    |    |-- aadress_ads__ads_normaliseeritud_taisaadress: string (nullable = true)
 |    |    |    |-- aadress_ads__ads_normaliseeritud_taisaadress_tapsustus: string (nullable = true)
 |    |    |    |-- aadress_ads__ads_oid: string (nullable = true)
 |    |    |    |-- aadress_ads__koodaadress: string (nullable = true)
 |    |    |    |-- aadress_ads__tyyp: string (nullable = true)
 |    |    |    |-- algus_kpv: string (nullable = true)
 |    |    |    |-- ehak: string (nullable = true)
 |    |    |    |-- ehak_nimetus: string (nullable = true)
 |    |    |    |-- kaardi_nr: long (nullable = true)
 |    |    |    |-- kaardi_pi

In [ ]:
r.printSchema()

NameError: ignored

In [ ]:
yldandmed_schema.printSchema()

root
 |-- ariregistri_kood: long (nullable = true)
 |-- nimi: string (nullable = true)
 |-- yldandmed: struct (nullable = true)
 |    |-- aadressid: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- aadress_ads__adob_id: string (nullable = true)
 |    |    |    |-- aadress_ads__adr_id: long (nullable = true)
 |    |    |    |-- aadress_ads__ads_normaliseeritud_taisaadress: string (nullable = true)
 |    |    |    |-- aadress_ads__ads_normaliseeritud_taisaadress_tapsustus: string (nullable = true)
 |    |    |    |-- aadress_ads__ads_oid: string (nullable = true)
 |    |    |    |-- aadress_ads__koodaadress: string (nullable = true)
 |    |    |    |-- aadress_ads__tyyp: string (nullable = true)
 |    |    |    |-- algus_kpv: string (nullable = true)
 |    |    |    |-- ehak: string (nullable = true)
 |    |    |    |-- ehak_nimetus: string (nullable = true)
 |    |    |    |-- kaardi_nr: long (nullable = true)
 |    |    |    |-- kaardi_pi

In [ ]:
r.show(10, truncate=False)

+----------+-----------------------------------------------------+-----+
|emtak_kood|emtak_tekstina                                       |count|
+----------+-----------------------------------------------------+-----+
|8622      |Eriarstiabi                                          |36   |
|01501     |Segapõllumajandus                                    |3356 |
|73201     |Turu-uuringud ja avaliku arvamuse küsitlused         |225  |
|46422     |Jalatsite hulgimüük                                  |17   |
|2932      |Mootorsõidukite  muude osade ja lisaseadmete tootmine|1    |
|46901     |Spetsialiseerimata hulgikaubandus                    |3913 |
|2229      |Muude plasttoodete tootmine                          |7    |
|66191     |Finantsnõustamine                                    |664  |
|10841     |Maitseainete ja -kastmete tootmine                   |28   |
|35221     |Maagaasi ülekanne ja  jaotus maagaasivõrgu  kaudu    |8    |
+----------+---------------------------------------

In [ ]:
for key in list(generic_data.keys()):
  path = file_path_unpacked(generic_data[key])
  s = "SELECT * FROM {key} limit 1".format(key=key)
  r = spark.sql(s)
  print(key)
  print(r.show())

lihtandmed
+---------------+----------------+-----------------------+---------------------------------+-------+-----------------+--------------------------+-----------------------+-----------------+---------------------------+-----------------+---------------------+--------------------------+----------+-----------+-------------------------------+--------------------+
|           nimi|ariregistri_kood|ettevotja_oiguslik_vorm|ettevotja_oigusliku_vormi_alaliik|kmkr_nr|ettevotja_staatus|ettevotja_staatus_tekstina|ettevotja_esmakande_kpv|ettevotja_aadress|asukoht_ettevotja_aadressis|asukoha_ehak_kood|asukoha_ehak_tekstina|indeks_ettevotja_aadressis|ads_adr_id|ads_ads_oid|ads_normaliseeritud_taisaadress|  teabesysteemi_link|
+---------------+----------------+-----------------------+---------------------------------+-------+-----------------+--------------------------+-----------------------+-----------------+---------------------------+-----------------+---------------------+----------------

In [ ]:
spark.stop()